![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Exercise8B-InSARTimeSeriesGIANTProcessing

<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />

### Franz J Meyer & Joshua J C Knicely; University of Alaska Fairbanks

The primary goal of this lab is to demonstrate how to process InSAR data, specifically interferograms, using the Generic InSAR Analysis Toolbox ([GIAnT](http://earthdef.caltech.edu/projects/giant/wiki)) in the framework of *Jupyter Notebooks*. GIAnT takes multiple connected InSAR-based surface deformation measurements as input and estimates the deformation time series relative to the first acquisition time.

**Our specific objectives for this lab are to:**

- Learn how to prepare data for GIAnT. 
- Use GIAnT to create maps of surface deformation. 

<img style="padding:7px;" src="NotebookAddons/OpenSARlab_logo.svg" width="170" align="right" />

<div class="alert alert-block alert-info">
<font face="Calibri">
<img style="padding: 7px" src="NotebookAddons/sierranegra.jpg"  width=300  align="right"><font size="5"> <b> Volcanic deformation: Sierra Negra, Galápagos Islands </b> </font> <br>

<font size="3"> We will use time series of InSAR data to analyze surface deformation at Sierra Negra, a highly active shield volcano in the Galapagos Islands. It is located in the south of Isabela Island, approximately 40 km west of Cerro Azul, which we studied in the previous lab.
    
The most recent eruption occurred from 26 June to 23 August 2018. We will be looking at the deformation before and during the eruption (picture by Benjamin Ayala), using Sentinel-1 data.
<br><br>

Over the course of more than two months, <a href="https://volcano.si.edu/volcano.cfm?vn=353050&vtab=Weekly">multiple fissures opened</a>. Lava that emerged from the fissures covered several tens of square kilometers. One lava flow reached the coastline on 6 July. 
<br><br>
</font>
</font>
</div>

## 0. Overview

**About GIAnT:**

GIAnT is a Python framework for performing InSAR time series analysis. It is capable of performing several types of Small BAseline Subset (SBAS) processing workflows. It also includes simple filtering approaches for separating deformation from tropospheric phase contributions.

**Limitations:**

GIAnT has a number of limitations that are important to keep in mind as these can affect its effectiveness for certain applications. It implements the simplest time-series inversion methods. Its single coherence threshold is very conservative in terms of pixel selection. It does not include any consistency checks for unwrapping errors. It has a limited dictionary of temporal model functions. It has limited capabilities for removing tropospheric errors.

**Using GIAnT:**

GIAnT requires very specific input. Because of the input requirements, the majority of one's effort goes to getting the data into a form that GIAnT can manipulate and to creating files that tell GIAnT what to do.

<div class="alert alert-warning">
    <b>GIAnT processing steps</b>
                            
   1. Prepare Data: Process interferograms and convert to GIAnT-friendly format<br><br>
   2. Run GIAnT to estimate deformation and mitigate atmospheric impacts<br><br>
   3. Data visualization and interpretation
</div>

Here, we will focus on the actual processing and visualization. The preparation steps have been completed for you in order to save disk space and computation time. The code to create the preparatory files has been included in the Exercise8A-InSARTimeSeriesGIAnTPreparation notebook. More information about GIAnT can be found here: http://earthdef.caltech.edu/projects/giant/wiki.

**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## 1. Import Python Libraries:

**Import the Python libraries and modules we will need to run this lab:**

In [ ]:
%%capture
from datetime import date
from pathlib import Path
import h5py # for is_hdf5
import shutil

import numpy as np
from osgeo import gdal
gdal.UseExceptions()


import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.dates
from matplotlib import rc


from IPython.display import HTML

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

**Download GIAnT from the `asf-jupyter-data-west` S3 bucket**

GIAnT is no longer supported (Python 2). This unofficial version of GIAnT has been partially ported to Python 3 to run this notebook. Only the portions of GIAnT used in this notebook have been tested.

In [ ]:
giant_path = Path("/home/jovyan/.local/GIAnT/SCR")

if not giant_path.parent.exists():
    download_path = 's3://asf-jupyter-data-west/GIAnT_5_21.zip'
    output_path = f"/home/jovyan/.local/{Path(download_path).name}"
    !aws --region=us-west-2 --no-sign-request s3 cp $download_path $output_path
    if Path(output_path).is_file():
        !unzip $output_path -d /home/jovyan/.local/
        Path(output_path).unlink()

## 2. Data preparation for GIAnT

Ordinarily, the first step of any SBAS analysis would consist of processing individual interferogram pairs. The rationale of SBAS (Short BAseline Subset) is to choose those pairs for which a high coherence can be expected. These are typically those pairs with a short temporal (horizontal axis) and spatial (vertical axis; less important for Sentinel-1 because they are always small) baselines. However, all these preparation steps have already been accomplished. 

The prepared data cube that consists of the stack of unwrapped interferograms and several other required files have been created and stored on a server. We will download this data to a local directory and unzip it. 

Before we download anything, **create a working directory for this analysis:**

In [ ]:
path = Path("/home/jovyan/notebooks/SAR_Training/English/Hazards/CBCInSAR")
data_path = path/'data_ts' 

if not path.exists():
    path.mkdir()
    
    if not data_path.exists():
        data_path.mkdir()

**Copy the zip file to your data directory:**

In [ ]:
!aws --region=us-west-2 --no-sign-request s3 cp s3://asf-jupyter-data-west/Lab9Files.zip .
!mv Lab9Files.zip InSARSierraNegra.zip

**Create the directories where we will perform the GIAnT analysis and store the data:**

In [ ]:
stack_path = data_path/"Stack"

if not stack_path.exists():
    stack_path.mkdir()

**Extract the zipped file to path and delete it:**

In [ ]:
zipped = Path('InSARSierraNegra.zip')

if zipped.exists():
    asfn.asf_unzip(str(path), str(zipped))
    zipped.unlink()

**Move the unzipped files into the Stack folder**

In [ ]:
temp_dir = path/'Lab9Files'

if not (stack_path/'RAW-STACK.h5').exists():
    shutil.move(str(temp_dir/'RAW-STACK.h5'), stack_path)

files = list(temp_dir.rglob('*.*'))
for file in files:
    if file.exists():
        try:
            shutil.move(str(file), data_path)
        except:
            pass
        
if temp_dir.exists():
    shutil.rmtree(temp_dir)

**Data preparation from scratch *(OPTIONAL)***

If you want to use a different stack of interferograms than those provided, please refer to the InSARTimeSeriesGIANTPreparation notebook for more information on how to prepare the unwrapped interferograms that a pair-wise processor (e.g., SNAP, ISCE, GAMMA) produces for GIAnT. 

## 3. Run GIAnT

We have now created all of the necessary files to run GIAnT

<div class="alert alert-block alert-warning">
<b>GIAnT workflow</b>
<br><br>
There are three functions that need to be called. The first one reads in all the previously prepared data and stores them in the RAW-STACK.h5 file. This file has already been created.
<br><br>
<ol>
    <li> <font  color="LightSlateGrey"><b>More data preparation: </b>PrepIgramStack.py</font></li>
    <li><b>Phase ramp removal: </b>ProcessStack.py</li>
    <li><b>Phase inversion and deformation estimation</b>: SBASInvert.py</li>
    </ol>
</div>

**Run PrepIgramStack.py (in our case, this has already been done):**

In [ ]:
#!python $giant_path/PrepIgramStack.py

### 3.1 Confirm That The Stack is an HDF5 File and Declare Parameters

PrepIgramStack.py creates a file called 'RAW-STACK.h5'. **Verify that RAW-STACK.h5 is an HDF5 file as required by the rest of GIAnT.**

In [ ]:
raw_h5 = stack_path/'RAW-STACK.h5'

if not h5py.is_hdf5(raw_h5):
    print(f"Not an HDF5 file: {raw_h5}")
else:
    print(f"Confirmed: {raw_h5} is an HDF5 file.")

**Set up parameters**

A range of parameters that are needed for the subsequent two processing steps need to be set. We will focus on the atmospheric filtering parameter. 

<div class="alert alert-block alert-warning">
<b>Filtering parameter</b>
<br><br>
GIAnT smoothes the inverted phase history in time. The degree of smoothing is determined by a filter parameter <b>filt</b>, given in the unit of [years].

<img src='NotebookAddons/filter.png' align='center' width=500>

A value of 0.5 (6 months) implies that any component (due to deformation, the atmosphere, etc.) that happens on a time scale of less than approximately 6 months will be smoothed out. 
</div>

GIAnT requires that the parameters be stored in an XML file sbas.xml. We already provided you with a functional sbas.xml. **Thus, you do not have to execute the code below, which overwrites the file. The only thing the code does is change the filtering parameter in the file.**

In [ ]:
############
# OPTIONAL #
############

filt = 0.5 # in years, change to vary the degree of smoothing

### change the parameter

fnxml = next(giant_path.parent.rglob('sbas.xml'))
fnxmlbu = giant_path.parent/'sbas_backup.xml'

# make a backup copy

if not fnxmlbu.exists():
    shutil.copyfile(fnxml, fnxmlbu)

# read the xml file
import xml.etree.ElementTree as ET
tree = ET.parse(fnxml)
root = tree.getroot()

# find the element we need
filter_element = root[0].find('filterlen').find('value')
# overwrite its content
filter_element.text = f'{filt}'
# store as xml
with open(fnxml, 'wb') as f:
    f.write(ET.tostring(root))

**Run this cell to retrieve the filter size if you opted to not run the code cell above**

Note: If you ran the optional cell above, running this one isn't necessary but doing so won't hurt anything.

In [ ]:
### change the parameter
fnxml = next(giant_path.parent.rglob('sbas.xml'))
fnxmlbu = giant_path.parent/'sbas_backup.xml'

# make a backup copy

if not fnxmlbu.exists():
    shutil.copyfile(fnxml, fnxmlbu)

# read the xml file
import xml.etree.ElementTree as ET
tree = ET.parse(fnxml)
root = tree.getroot()
# find the element we need
filter_element = root[0].find('filterlen').find('value')
filt = filter_element.text

### 3.2 Ramp removal: ProcessStack.py

<div class="alert alert-warning">
<font face='Calibri' size='3'> <img src="NotebookAddons/deramped.png" align="right" width=400></font>
<br>   
<font face='Calibri' size='3'>
<b>Ramps in the interferogram</b><br>
Interferograms can often have pronounced ramps. One origin of such ramps are slightly inaccurate baseline estimates due to uncertainty in the satellite orbits. It is thus common to remove ramps from the interferogram, in particular if one is interested in localized deformation. We will use the standard ramp removal implemented in GIAnT (netramp=True in our sbas.xml). Owing to the excellent accuracy of the orbits used, we do not expect major ramps. Also, we are looking at a relatively confined area. <br><br>

<b>Large-scale atmospheric corrections</b> (not done here)<br>
It is also possible to use a priori weather model data to mitigate predictable large-scale tropospheric phase patterns. There is an additional option for estimating and removing stratified tropospheric contributions. We will not take advantage of these advanced processing options in this tutorial. <br><br>

<b>Implementation in ProcessStack.py</b><br>
To remove ramps, we call ProcessStack.py. It produces a file called PROC-STACK.h5, on which the later processing (phase inversion and deformation estimation) operates.  
</font> <br>
</div>

**Run ProcessStack.py:**

Note: The progress bars may not fully complete. This is okay.

In [ ]:
with asfn.work_dir(data_path):
  !python {giant_path/'ProcessStack.py'}

<font face='Calibri' size='4'> <b>3.3 Phase inversion and deformation estimation: Run SBASInvert.py </b></font>
<br>
<font face='Calibri' size='3'> SBAS Invert.py takes the PROC-STACK.h5 file produced by the previous step and estimates the deformation time series. It incorporates two steps:
<br>
<div class="alert alert-warning">
    <b> Obtaining the raw and smoothed phase history</b><br>
    
   <b>1. Phase inversion to obtain the raw estimate: </b> 
    Solve the least-squares problem to get a best-fit phase history from the interferogram phases. We refer to this as the raw estimate. No constraints (and temporal model) are assumed for the phase history: each scene phase is a separate unknown in the estimation. The inversion enforces a least-norm constraint whenever there are multiple non-connected clusters of interferograms.<br><br>
   <b>2. Temporal smoothing:</b>
      As the raw phase history still contains noise (e.g. from the atmospheric phase screen), the inverted phase history is smoothed.
    The default choice for the temporal smoothing that we use in this lab is very strong (i.e. a long filter length). The choice of smoothing to mitigate artefacts due to e.g. the troposphere is a critical one. We will analyze the choice in detail later. 
</div>

The output, most importantly the line-of-sight deformation time series, is stored in the file LS-PARAMS.h5.
</font>


<font face='Calibri' size='3'><b>Run SBASInvert.py:</b>
<br>
Note: The progress bar may not fully complete. This is okay.</font>

In [ ]:
with asfn.work_dir(data_path):
  !python {giant_path/'SBASInvert.py'}

## 4. Data Visualization

To explore our results, we will now produce a number of plots.

<div class="alert alert-success">
<b>Interpreting the results</b>
<br><br>
Each plot will be followed by a question that will let you explore and interpret a particular aspect of the results. 
</div>

**We first create a folder in which to store the figures:**

In [ ]:
plot_dir = path/'plots'

if not plot_dir.exists():
    plot_dir.mkdir()

**Load the stack produced by GIAnT and read it into an array so we can manipulate and display it:**

In [ ]:
# params_h5 = f"{stack_path}/LS-PARAMS.h5"
params_h5 = stack_path/"LS-PARAMS.h5"
f = h5py.File(str(params_h5), 'r')
data_cube = f['recons'][()] # filtered deformation time series
data_cube_raw = f['rawts'][()] # raw (unfiltered) deformation

**Read and convert the dates:**

In [ ]:
dates = list(f['dates']) # these dates appear to be given in Rata Die style: floor(Julian Day Number - 1721424.5). 
tindex = [date.fromordinal(int(d)) for d in dates]

### 4.1 Amplitude image:

In [ ]:
plt.rcParams.update({'font.size': 14})
radar_tiff = data_path/'20161119-20170106_amp.tiff'
radar=gdal.Open(str(radar_tiff))
im_radar = radar.GetRasterBand(1).ReadAsArray()
radar = None
dbplot = np.ma.log10(im_radar)
vmin=np.percentile(dbplot.data, 3)
vmax=np.percentile(dbplot.data, 97)
fig = plt.figure(figsize=(16,7)) # Initialize figure with a size
ax1 = fig.add_subplot(111) # 221 determines: 2 rows, 2 plots, first plot
ax1.imshow(dbplot, cmap='gray',vmin=vmin,vmax=vmax,alpha=1);
plt.title('Amplitude [logarithmic]')
plt.grid()
fnfig = plot_dir/'SierraNegra-dBScaled-AmplitudeImage.png'
plt.savefig(fnfig,dpi=200,transparent='false')

<div class="alert alert-success">
<img style="padding: 7px" src="NotebookAddons/calderasierranegra.jpg" align="right" width=300>
    
<font face='Calibri' size='3'><b>Studying the caldera</b>


The caldera, the large circular depression in the central portion of the image, formed due to the collapse of a magma chamber in a previous eruption. 
    
Compare the photograph (from Nature Galapagos) with the SAR image: 
- Try to see which parts correspond to each other (Hint: also look at the background). 
- Can you infer the radar look direction from this figure? 
    </font>
<br><br>
</div>

### 4.2 Deformation map:

**We will first write a helper function that produces the plot given a cumulative deformation estimate:**

In [ ]:
def defNradar_plot(deformation, radar, title="Cumulative deformation [mm]"):
    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(111)
    vmin = np.percentile(radar.data, 3)
    vmax = np.percentile(radar.data, 97)
    ax.imshow(radar, cmap='gray', vmin=vmin, vmax=vmax)
    fin_plot = ax.imshow(deformation, cmap='RdBu', vmin=-50.0, vmax=50.0, alpha=0.75)
    fig.colorbar(fin_plot, fraction=0.24, pad=0.02)
    ax.set(title=title)
    plt.grid()

Now we call the function. The scene variable is set to -1 (the last one), so that the cumulative deformation over the entire period is plotted. You can also change it to a smaller value.

In [ ]:
# Choose scene for which to plot deformation
scene = data_cube.shape[0] - 1  # try any number between 0 and 45

# Make a nice title for the figure
title = f'Cumulative deformation [mm] {tindex[0].strftime("%Y-%m-%d")} to {tindex[scene].strftime("%Y-%m-%d")}'

# Get deformation map and radar image we wish to plot
deformation = data_cube[scene, ...]

# Call function to plot an overlay of our deformation map and radar image.
defNradar_plot(deformation, dbplot, title=title)
fnfig = plot_dir/'SierraNegra-DeformationComposite.png'
plt.savefig(fnfig, dpi=200, transparent='false')

<div class="alert alert-success">
<font face='Calibri' size='3'>    
<b>Location and direction of the deformation</b>
<br><br>
The deformation map shows a clear contrast between the rim of the caldera and the the rest of the map. The sign convention is such that positive values correspond to a movement toward the satellite. Did the area near the caldera move up or down? What about the gray areas?
</font>
</div>

### 4.3 Deformation time series:

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
plt.grid()

# choose a stable (?) point
point = (300, 50) # first axis is vertical, second axis horizontal, (0,0) upper left

l1 = ax.plot(tindex, data_cube[:, point[0], point[1]], label='Filtered')
l2 = ax.plot(tindex, data_cube_raw[:, point[0], point[1]], label='Not filtered')
ax.legend()
ax.xaxis.set_major_locator(matplotlib.dates.MonthLocator(bymonth=[1,7]))
ax.set_title('Comparing filtered and unfiltered solution')
fnfig = plot_dir/'SierraNegraTimeSeries.png'
plt.savefig(fnfig, transparent=False)

<div class="alert alert-success">
<font face='Calibri' size='3'>
<b>Atmospheric filtering</b>
<br><br>
GIAnT applied a very strong temporal smoothing filter to the data. The idea was to reduce the noise, in particular that due to the troposphere. Do you think the smoothing was adequate? Do you think the difference between the filtered and the unfiltered time series is entirely due to noise, or could some of the discrepancy be due to temporally variable deformation. What time period could most plausibly be affected by a transient deformation signal that got lost due to the smoothing?    
</font>
</div>

### 4.4 Spatiotemporal deformation - Animation:

**First, write a function to create an animation:** 

In [ ]:
def create_animation(deformation=data_cube):
    fig = plt.figure(figsize=(14, 8))
    ax = fig.add_subplot(111)
    ax.axis('off')
    vmin=np.percentile(deformation.flatten(), 5)
    vmax=np.percentile(deformation.flatten(), 95)


    im = ax.imshow(deformation[-1, ...], cmap='RdBu', vmin=-50.0, vmax=50.0)
    ax.set_title("Cumulative deformation until {} [mm]".format(tindex[-1]))
    fig.colorbar(im)
    plt.grid()

    def animate(i):
        ax.set_title("Cumulative deformation until {} [mm]".format(tindex[i]))
        im.set_data(deformation[i])

    ani = matplotlib.animation.FuncAnimation(fig, animate, frames=deformation.shape[0], interval=400)
    return ani

**We will animate the filtered time series.**

If you uncomment the third line, you can animate the raw unfiltered time series.

In [ ]:
%%capture
type_time_series = 'filtered'
# type_time_series = 'not_filtered'

if type_time_series == 'filtered':
    ani = create_animation(deformation=data_cube)
else:
    ani = create_animation(deformation=data_cube_raw)

**Create a javascript animation of the time-series running inline in the notebook and save it as .gif:**

In [ ]:
rc('animation', embed_limit=10.0**9)
fnani = plot_dir/f'SierraNegraDeformationTS{type_time_series}_{filt}.gif'
ani.save(fnani, writer='pillow', fps=2)
HTML(ani.to_jshtml())

<div class="alert alert-success">
<font face='Calibri' size='3'>
<b>Atmospheric filtering II</b>
<br><br>
Compare the animations for the filtered and non-filtered time series. Where and when are the differences largest? 

<b>Reprocess the data</b> using a shorter filter (e.g. one month: filt = 0.082) and compare the differences.
</font>
</div>

*Exercise8B-InSARTimeSeriesGIANTProcessing.ipynb - Version 1.5.0 - November 2021*

*Version Changes:*

- *asf_notebook -> opensarlab_lib*
- *html -> markdown*
- *url_widget*